Выполнила команда: __Терлыч Никита__, __Сунцев Максим__

# Описание лучшей модели

Использованые фичи:

- Категориальные: ['category_id','sold_mode', 'product_type', 'payment_available', 
                    'subcategory_id', 'city', 'delivery_available', 'img_num', 'region']
- Непрерывные: ['lat', 'long','price']
- Текстовые: ['desc_text', 'name_text']

Категориальные фичи были преобработаны и заменены на "вероятности" относительно target фичи.

Текстовые данные обрабатывались в отдельном пайплайне, включающем в себя:

1. Токенизацию - CountVectorizer()
2. Нормализацию со взвешиванием по tf-idf - TfidfTransformer()
3. Предсказание классов через SGDClassifier(loss='log', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)

На непрерывных и категориальных данных обучались два классификатора: RandomForestClassifier и CatBoostClassifier.
Для получения предсказания на тестовых данных, предсказания всех классификаторов (SGDClassifier, RandomForestClassifier и CatBoostClassifier) работали в ансамбле, где они взвешивались и усреднялись, а среднее значение выводилось как ответ.

Для оценки модели имеющийся тренировачный датасет был подразбит на test и train для тренироваки модели.
Качество модели оценивалось площадью ROC-кривой и метрикой точности предсказаний.

## Ниже приведены блоки кода лучшего из решений:


In [13]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import nltk
nltk.download('stopwords')
import string

class Solver:
    """
        Класс принимает обучающую и тестовую выборки данных
        и решает задачу классификации при вызове метода solve()
    """

    def __init__(self, data_train, data_test):
        """
            Вызывает методы препроцессинга данных
        """
        self.data = data_train
        self.data_test = data_test
        self.prepare_data()
        self.prepare_test()
    
    def prepare_data(self):
        """
            Препроцессинг обучающего датасета
        """
        self.X_train = self.data[['lat', 'long','price']].values
        self.y_train = self.data['sold_fast']
        self.cat_features = ['category_id','sold_mode', 'product_type', 'payment_available', 'subcategory_id', 
                             'city', 'delivery_available', 'img_num', 'region']
        self.cat_features_dict = self.preprocess_cat_features('sold_fast')
        for feature in self.cat_features:
            res = [0] * len(self.data)
            for i, val in enumerate(self.data[feature].values):
                res[i] = self.cat_features_dict[feature][val]
            self.X_train = np.c_[self.X_train, np.array(res)]

    def prepare_test(self):
        """
            Препроцессинг тестового датасета
        """
        X_test = self.data_test[['lat', 'long','price']].values #'lat', 'long', 
        for feature in self.cat_features:
            res = [0] * len(self.data_test)
            for i, val in enumerate(self.data_test[feature].values):
                res[i] = self.cat_features_dict[feature].get(val, 0)
            X_test = np.c_[X_test, np.array(res)]
        self.X_test = X_test
    
    def target_encoding(self, features, targets):
        """
            Инкодинг категориальных фич (унаследовано с семинара)
        """
        values = defaultdict(int)
        counts = Counter()
        for val, target in zip(features, targets):
            values[val] += target
            counts[val] += 1

        mean_values = dict()
        for val in values:
            mean_values[val] = values[val] / counts[val]
        return mean_values

    def preprocess_cat_features(self, target):
        """
            Запускает инкодинг категориальных фич (унаследовано с семинара)
        """
        self.cat_features_dict = dict()
        for feature in self.cat_features:
            self.cat_features_dict[feature] = self.target_encoding(self.data[feature].values, self.data[target].values)
        return self.cat_features_dict
    
    def ensemble_proba(self):
        """
            Самописный ансамбль: считает предсказания для класса [1] по всем классификаторам
            и взвешенное среднее значение для каждого предсказания
        """
        probas = []
        df = pd.DataFrame()
        # Random Forest Classifier
        df['rt'] = self.rt_proba(n=100)
        # Cat Boost Classifier
        df['catboost'] = self.catboost_proba()
        # Классификация текстовых фич
        df['textCF'] = self.textCF_proba(self.data['desc_text'], self.data['sold_fast'], self.data_test['desc_text'])
        df['nameCF'] = self.textCF_proba(self.data['name_text'], self.data['sold_fast'], self.data_test['name_text'])
        # Переменная для анализа/дебага результатов вне класса 
        self.arr = df.values
        return np.average(self.arr, axis = 1, weights = [0.2,0.2,0.7,0.5])
    
    def ensemble_score(self, y_test):
        """
            Печатает точность предсказаний для каждого класса из ансамбля
            (используется для дебага и тюнинга параметров)
        """
        rt = self.rt_proba(n=10, predict = True)
        catboost = self.catboost_proba(predict = True)
        text = self.textCF_proba(self.data['desc_text'], self.data['sold_fast'], self.data_test['desc_text'], predict = True)
        name = self.textCF_proba(self.data['name_text'], self.data['sold_fast'], self.data_test['name_text'], predict = True)
        from sklearn.metrics import accuracy_score, roc_auc_score
        print("RT acc = {}".format(accuracy_score(y_test, rt)))
        print("CatBoost acc = {}".format(accuracy_score(y_test, catboost)))
        print("TextCF acc = {}".format(accuracy_score(y_test, text)))
        print("NameCF acc = {}".format(accuracy_score(y_test, name)))
        
    def rt_proba(self, n=50, predict = False):
        """
            Считает probability для Random Forest классификатора
            либо предсказания, если параметр predict == True
            Args:
                n - int, количество деревьев в лесу
                predict - bool, возвращать точные предсказания или вероятность
        """
        rt = RandomForestClassifier(max_depth=15, n_estimators=n, random_state=43)
        rt.fit(self.X_train, self.y_train)
        if predict:
            return rt.predict(self.X_test)
        return rt.predict_proba(self.X_test)[:,1]
    
    def textCF_proba(self, X_train, y_train, X_test, predict = False):
        """
            Считает probability для классификатора текстовых данных
            либо предсказания, если параметр predict == True
            Args:
                X_train, y_train, X_test - массивы данных для классификатора
                predict - bool, возвращать точные предсказания или вероятность
        """
        textCF = TextCF()
        textCF.fit(X_train, y_train)
        if predict:
            return textCF.predict(X_test)
        text_proba = textCF.predict_proba(X_test)
        return text_proba[:,1]
    
    def catboost_proba(self, predict = False):
        """
            Считает probability для CatBoost классификатора
            либо точные предсказания, если параметр predict == True
            Args:
                predict - bool, возвращать точные предсказания или вероятность
        """
        model = CatBoostClassifier(iterations=1000,
                          depth=6, eval_metric='AUC', od_type = 'IncToDec',
                                  logging_level='Silent')
    
        model.fit(self.X_train, self.y_train)
        if predict:
            return model.predict(self.X_test)
        return model.predict_proba(self.X_test)[:,1]
        
    def solve(self):
        """
            Решает поставленную задачу классификации
            и записывает результат в файл для отправки
        """
        self.proba = s.ensemble_proba()
        product_id = self.data_test['product_id'].values
        data = pd.DataFrame.from_dict({'product_id' : product_id, 'score' : self.proba})
        data.to_csv('./to_submit', sep = ',', index = False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

class TextCF:
    """
        Класс обёртка для пайплайна обработки и классификации текстовых данных
        (когда-то был более полным и сложным, но в итоге остался просто обёрткой)
    """
    def __init__(self):
        self.text_clf = Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='log', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
                ])
        
    def fit(self, X_train, y_train):
        self.text_clf.fit(X_train, y_train) 

    def predict(self, X_test):
        self.predicted = self.text_clf.predict(X_test)
        return self.predicted        
    
    def predict_proba(self, X_test):
        self.proba = self.text_clf.predict_proba(X_test)
        return self.proba

In [5]:
# Чтение датасетов
data = pd.read_csv('./train.tsv', sep = '\t')
data_test = pd.read_csv('./test_nolabel.tsv', sep = '\t')

In [6]:
# Тестовые данные
from sklearn.metrics import accuracy_score, roc_auc_score
train, X_test, y_test = data.iloc[:250000, :], data.iloc[250000:, :-1], data.iloc[250000:, -1]

In [88]:
%%time
# Блок для дебага классификаторов по отдельности
s = Solver(data, data_test)
arr = s.catboost_proba()
print("ROC-AUC = {}".format(roc_auc_score(y_test, arr)))
# 0.6163551452254086 - # depth 6, 1000 trees - 1min 43s

Wall time: 2min 3s


In [8]:
%%time
# Блок для дебага ансамбля
s = Solver(train, X_test)
arr = s.ensemble_proba()
print("ROC-AUC = {}".format(roc_auc_score(y_test, arr)))
# 0.627541738379946

C:\ProgramData\Miniconda3\envs\jupyterlab-ext\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


ROC-AUC = 0.6218277184031248
Wall time: 57.3 s


In [10]:
# Блок для подбора весов руками
arr = np.average(s.arr, axis = 1, weights= [0.2,0.2,0.7,0.5])
print("ROC-AUC = {}".format(roc_auc_score(y_test, arr)))
# 0.6222343832877966 [0.2,0.2,0.7,0.5]

ROC-AUC = 0.6218277184031248


In [11]:
%%time 
# Блок тестирования точности предсказаний классов
s = Solver(train, X_test)
arr = s.ensemble_score(y_test)

C:\ProgramData\Miniconda3\envs\jupyterlab-ext\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


RT acc = 0.766540614725368
CatBoost acc = 0.7671527729781499
TextCF acc = 0.7683869630039198
NameCF acc = 0.7683869630039198
Wall time: 22.2 s


In [ ]:
%%time
# Блок записи решения в файл для отправки
s = Solver(data, data_test)
s.solve()

# Описание альтернативных подходов/попыток
В процессе решения задачи были выполнены попытки учесть все доступные фичи, но не все удалось включить в модель.
Например, для фичи 'properties' было неочевидно, что лучше распарсить и учитывать даннеы как текст, либо как категории; первые попытки учесть эти данные не улучшили модель, и идея была заброшена.

Фичу 'date', наоборот, было легко распарсить, после чего внедрить в датасет день и месяц добавления товара, однако результатом было переобучение модели, с которым не было возможности справиться вовремя, и фича была удалена из датасета.

Набор классификаторов претерпел много изменений, в модели были перепробованы буквально все классификаторы, доступные в библиотеке scikit-learn, но лучшими оказались RandomForest и SGD. Позже к ним был добавлен CatBoost. После объединения всех класификаторов в ансамбль, подбирался вес "голоса" для каждого из них после любой модификации или подкрутки параметров так, чтобы основные метрики были максимальны.

Многие результаты изменений моделей не сохранились, однако некоторые логи всё же остались во вспомагательных ячейках:
Тюнинг CatBoost'а:
- ROC: 0.6119392482947758 - depth = 2
- ROC: 0.6155439202758909 - depth = 6
- ROC: 0.6159188915313375 - 500 trees
- ROC: 0.6163551452254086 - 1000 trees

Ансамбль с весами и без: 
- ROC: 0.6209800843158668 [1, 1, 1, 1]
- ROC: 0.6222343832877966 [0.2, 0.2, 0.7, 0.5]